In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from QuantConnect.Research import QuantBook

processed_symbols = []
qb = QuantBook()

In [ ]:
from math import ceil,floor
import datetime as dt
import pandas as pd
import numpy as np
from sklearn import linear_model

In [ ]:
def prophet_trendline_slope(number_of_forecast_days:int=1):
    history = qb.History(stock.Symbol, 22, Resolution.Daily)
    df = history.loc[stock.Symbol].reset_index()[["time","close"]].rename(columns = {"time":"ds","close":"y"})
    assert list(df.columns) == ["ds", "y"], print("dataframe columns must adhere to predefined column scheme.")
    m = Prophet(changepoint_prior_scale=0.05,changepoint_range=1)
    m.fit(df)
    future = m.make_future_dataframe(periods=number_of_forecast_days)
    forecast = m.predict(future)

    slope_df = pd.DataFrame(columns=["date_ordinal","trend_value"])
    slope_df.date_ordinal = [i for i in range(22)]
    
    slope_df.trend_value = forecast.trend
    reg = linear_model.LinearRegression()
    reg.fit(slope_df['date_ordinal'].values.reshape(-1, 1), slope_df['trend_value'].values)
    return reg.coef_[0]   

In [ ]:
def created_model_and_predict(df, number_of_forecast_days):
    assert list(df.columns) == ["ds", "y"], print("dataframe columns must adhere to predefined column scheme.")
    m = Prophet(changepoint_prior_scale=0.05,changepoint_range=1)
    m.fit(df)
    future = m.make_future_dataframe(periods=number_of_forecast_days)
    forecast = m.predict(future)
    return m, forecast   


In [ ]:
def generate_analysis(symbol:str, 
                    asset_type:str,
                    primary_input_variable:str, 
                    past_data_days:int, 
                    bond_time_frame:int,
                    number_of_forecast_days:int,
                    componenets:bool,
                    risk_analysis:bool,
                    secondary_input_variable:str,
                    secondary_input_variable_moving_window:int=5,
                    slope_analysis:bool=True,
                    slope_moving_average:int=200,
                    slope_moving_window:int=5):

    asset_types = ["equity","bonds","futures"]
    primary_input_types = ["open", "close","high","low","volume"]
    custom_secondary_variables = ["velocity", "acceleration","standard_deviation"]
    list_of_avail_bond_timeframes = {2:"twoyear",5:"fiveyear",10:"tenyear",30:"thirtyyear"}
    futures_map = {"USD":Futures.Currencies.USD,
                   "Crude Oil": Futures.Energies.CrudeOilWTI,
                   "SPY Futures":Futures.Indices.SP500EMini,
                   "Gold":Futures.Metals.Gold}

    assert primary_input_variable in primary_input_types, print(f"Primary input type must be one of the following: {primary_input_types}/")
    assert asset_type in asset_types, print(f"Asset type must be one of the following: {asset_types}.")
    assert secondary_input_variable in custom_secondary_variables, print("Secondary variable doesn't exist in internal list.")
    assert secondary_input_variable_moving_window in [2,5,22], print("Secondary variable window should be 2, 5, or 22, corresponding to daily, weekly, and monthly.")
    
    if asset_type == "equity":
        if symbol not in processed_symbols:
            if symbol == "VIX":
                qb.AddData(CBOE, "VIX", Resolution.Daily)
            else:
                stock = qb.AddEquity(symbol)
            processed_symbols.append(symbol)
    elif asset_type == "bonds":
        assert bond_time_frame in [2,5,10,30], "Possible values: 2,5,10,30 years."
        bond_time_frame = list_of_avail_bond_timeframes[bond_time_frame]
        if symbol not in processed_symbols:
            qb.AddData(USTreasuryYieldCurveRate, symbol) #USTYCR
            processed_symbols.append(symbol)
    elif asset_type == "futures":
        if symbol not in processed_symbols:
            qb.AddFuture(futures_map[symbol])
            processed_symbols.append(symbol)
   

    history = qb.History(qb.Securities.Keys, past_data_days, Resolution.Daily)

    if asset_type == "futures":
        history = history.droplevel([0])
        list_of_ids = list(history.index.levels[1])
    else:
        list_of_ids = list(history.index.levels[0])

    if symbol == "VIX":
        current_identifier = "VIX.CBOE 2S"
    elif symbol == "Crude Oil":
        current_identifier = "CL JL"
    elif symbol == "Gold":
        current_identifier = "GC 1P9"
    elif symbol == "SPY Futures":
        current_identifier = "ES 1S1"
    elif symbol == "USD Futures":
        current_identifier = ""
    elif symbol == "EA":
        current_identifier = "ERTS R735QTJ8XC9X"
    elif symbol == "EIX":
        current_identifier = "EIX R735QTJ8XC9X"
    else:
        current_identifier = [identifier for identifier in list_of_ids if symbol in identifier][0]

    if asset_type == "bonds":
        df = history.loc[current_identifier][[bond_time_frame]]
        df.reset_index(inplace=True)
        df.rename(columns={"time":"ds", bond_time_frame:"y"},inplace=True)
    else:
        df = history.loc[current_identifier][[primary_input_variable]]
        df.reset_index(inplace=True)
        df.rename(columns={"time":"ds", primary_input_variable:"y"},inplace=True)

    if asset_type == "bonds":
        df.ds = pd.to_datetime(df.ds, utc=True,errors='coerce').dt.strftime('%Y-%m-%d')

    m, forecast = created_model_and_predict(df, number_of_forecast_days)

    # Price plot 
    price_plot = plot_plotly(m, forecast)
    print(f"{primary_input_variable} PLOT")
    price_plot.show()
    if componenets:
        print(f"{primary_input_variable} COMPONENTS PLOT")
        components_plot = plot_components_plotly(m, forecast) 
        components_plot.show()
    
    df_secondary = df.copy(deep=True)

    if "velocity" in secondary_input_variable or "acceleration" in secondary_input_variable:
        # Caluclate velocity first
        df_secondary["velocity"] =  df_secondary.y.diff(periods=secondary_input_variable_moving_window)/secondary_input_variable_moving_window
        df_secondary = df_secondary[["ds","velocity"]]
        df_secondary.dropna(inplace=True)

        if "acceleration" in secondary_input_variable:
            df_secondary["acceleration"] = df_secondary.velocity.diff(periods=secondary_input_variable_moving_window)/secondary_input_variable_moving_window
            df_secondary = df_secondary[["ds","acceleration"]]
            df_secondary.dropna(inplace=True)
            df_secondary.rename(columns={"acceleration":"y"}, inplace=True)
            m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
            print(f"{primary_input_variable} ACCELERATION PLOT")
            acceleration_plot = plot_plotly(m, forecast)
            acceleration_plot.show()

            if componenets:
                print(f" {primary_input_variable} ACCELERATION COMPONENTS PLOT")
                components_plot = plot_components_plotly(m, forecast) 
                components_plot.show()
        else:
            print(f"{primary_input_variable} VELOCITY PLOT")
            df_secondary.rename(columns={"velocity":"y"}, inplace=True)
            m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
            velocity_plot = plot_plotly(m, forecast)
            velocity_plot.show()

            if componenets:
                print(f"{primary_input_variable} VELOCITY COMPONENTS PLOT")
                components_plot = plot_components_plotly(m, forecast) 
                components_plot.show()
    
    df_secondary = df.copy(deep=True)

    if risk_analysis:
        print("RISK ANALYSIS")
        df_secondary["std"] =  df_secondary.y.rolling(window=secondary_input_variable_moving_window).std()
        df_secondary = df_secondary[["ds","std"]]
        df_secondary.dropna(inplace=True)
        df_secondary.rename(columns={"std":"y"}, inplace=True)
        m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
        print(f"{primary_input_variable} STANDARD DEVIATION PLOT")
        std_plot = plot_plotly(m, forecast)
        std_plot.show()

        if componenets:
            print(f"{primary_input_variable} STANDARD DEVIATION COMPONENTS PLOT")
            components_plot = plot_components_plotly(m, forecast) 
            components_plot.show()

        if "velocity" in secondary_input_variable or "acceleration" in secondary_input_variable:
            # Caluclate velocity first
            df_secondary["velocity"] =  df_secondary.y.diff(periods=secondary_input_variable_moving_window)/secondary_input_variable_moving_window
            df_secondary = df_secondary[["ds","velocity"]]
            df_secondary.dropna(inplace=True)

            if "acceleration" in secondary_input_variable:
                df_secondary["acceleration"] = df_secondary.velocity.diff(periods=secondary_input_variable_moving_window)/secondary_input_variable_moving_window
                df_secondary = df_secondary[["ds","acceleration"]]
                df_secondary.dropna(inplace=True)
                df_secondary.rename(columns={"acceleration":"y"}, inplace=True)
                m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
                print(f"{primary_input_variable} STANDARD DEVIATION ACCELERATION PLOT")
                acceleration_plot = plot_plotly(m, forecast)
                acceleration_plot.show()

                if componenets:
                    print(f"{primary_input_variable} STANDARD DEVIATION ACCELERATION COMPONENTS PLOT")
                    components_plot = plot_components_plotly(m, forecast) 
                    components_plot.show()
            else:
                print(f"{primary_input_variable} STANDARD DEVIATION VELOCITY PLOT")
                df_secondary.rename(columns={"velocity":"y"}, inplace=True)
                m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
                velocity_plot = plot_plotly(m, forecast)
                velocity_plot.show()

                if componenets:
                    print(f"{primary_input_variable} STANDARD DEVIATION VELOCITY COMPONENTS PLOT")
                    components_plot = plot_components_plotly(m, forecast) 
                    components_plot.show()

    df_secondary = df.copy(deep=True)

    if slope_analysis:
        print("SLOPE ANALYSIS")
        df_secondary["smax"] = df_secondary.y.rolling(window=slope_moving_average).mean().dropna()
        df_secondary["smax_slope"] = (df_secondary.smax.diff(slope_moving_window) / slope_moving_window)
        df_secondary = df_secondary[["ds","smax_slope"]]
        df_secondary.dropna(inplace=True)
        df_secondary.rename(columns={"smax_slope":"y"}, inplace=True)
        m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
        print(f"{primary_input_variable} SMA SLOPE PLOT")
        std_plot = plot_plotly(m, forecast)
        std_plot.show()

        if componenets:
            print(f"{primary_input_variable} SMA SLOPE STANDARD DEVIATION COMPONENTS PLOT")
            components_plot = plot_components_plotly(m, forecast) 
            components_plot.show()

        if "velocity" in secondary_input_variable or "acceleration" in secondary_input_variable:
            # Caluclate velocity first
            df_secondary["velocity"] =  df_secondary.y.diff(periods=secondary_input_variable_moving_window)/secondary_input_variable_moving_window
            df_secondary = df_secondary[["ds","velocity"]]
            df_secondary.dropna(inplace=True)

            if "acceleration" in secondary_input_variable:
                df_secondary["acceleration"] = df_secondary.velocity.diff(periods=secondary_input_variable_moving_window)/secondary_input_variable_moving_window
                df_secondary = df_secondary[["ds","acceleration"]]
                df_secondary.dropna(inplace=True)
                df_secondary.rename(columns={"acceleration":"y"}, inplace=True)
                m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
                print(f"{primary_input_variable} SMA SLOPE STANDARD DEVIATION ACCELERATION PLOT")
                acceleration_plot = plot_plotly(m, forecast)
                acceleration_plot.show()

                if componenets:
                    print(f"{primary_input_variable} SMA SLOPE STANDARD DEVIATION ACCELERATION COMPONENTS PLOT")
                    components_plot = plot_components_plotly(m, forecast) 
                    components_plot.show()
            else:
                print(f"{primary_input_variable} SMA SLOPE STANDARD DEVIATION VELOCITY PLOT")
                df_secondary.rename(columns={"velocity":"y"}, inplace=True)
                m, forecast = created_model_and_predict(df_secondary, number_of_forecast_days)
                velocity_plot = plot_plotly(m, forecast)
                velocity_plot.show()

                if componenets:
                    print(f"{primary_input_variable} SMA SLOPE STANDARD DEVIATION VELOCITY COMPONENTS PLOT")
                    components_plot = plot_components_plotly(m, forecast) 
                    components_plot.show()


In [ ]:
generate_analysis(symbol="TXN", 
                asset_type="equity", # equity, bonds, futures (not supported yet)
                primary_input_variable="close", # close, open, high, low, volume
                past_data_days=past_data_days, # Past data to fit
                bond_time_frame=None, # 2,5,10,30
                number_of_forecast_days=number_of_forecast_days, # Future forecast days
                componenets=True, # True or False
                risk_analysis=True, # Include standard deviation analysis
                secondary_input_variable="acceleration", # velocity, acceleration
                secondary_input_variable_moving_window=secondary_input_variable_moving_window,
                slope_analysis=True,
                slope_moving_average=slope_moving_average,
                slope_moving_window=slope_moving_window) # Moving window used in calculating secondary input variables (2,5,22) - daily, weekly, monthly

In [ ]:
generate_analysis(symbol="SPY Futures", 
                asset_type="futures", # equity, bonds, futures (not supported yet)
                primary_input_variable="close", # close, open, high, low, volume
                past_data_days=past_data_days, # Past data to fit
                bond_time_frame=None, # 2,5,10,30
                number_of_forecast_days=number_of_forecast_days, # Future forecast days
                componenets=True, # True or False
                risk_analysis=True, # Include standard deviation analysis
                secondary_input_variable="acceleration", # velocity, acceleration
                secondary_input_variable_moving_window=secondary_input_variable_moving_window) # Moving window used in calculating secondary input variables (2,5,22) - daily, weekly, monthly